# Generate Unrelaxed Surfaces
Follow the strategy described by [Lazaro et al.](https://www.sciencedirect.com/science/article/pii/S0039602804000421#FIG2) to make symmetric 2D slabs.

In [1]:
from pathlib import Path
from ase.db import connect
import numpy as np
import json

Configuration

In [2]:
facet: tuple[int, int, int] = (1, 0, 0)  # Which surface to use
supercell: int = 2  # Number of cells in the slab direction
target_layers: int = 9  # Number of unit layers for the slab (7 is what Lazaro found as sufficient)

In [3]:
assert facet == (1, 0, 0)  # Haven't implemented the others yet

Compute the number of times to repeat the unit cell.

There is always one additional layer added during 'slabification' and 2 layers per unit cell

In [4]:
repeats = np.ceil((target_layers - 1) / 2. / supercell).astype(int)
layers = repeats * 2 * supercell + 1
print(f'Repeating unit cell {repeats} times to create {layers} layers')

Repeating unit cell 2 times to create 9 layers


## Make function to create slabs
Following [Lazaro et al.](https://www.sciencedirect.com/science/article/pii/S0039602804000421#FIG2), make two slabs: one with the AO2 and the other with a BO2 surfaces.

In [5]:
def slabify(atoms, z_duplicates, make_symmetric):
    """Add a vacuum layer, then copy the bottom layer up to the top

    Args:
        atoms: Initial supercell
        z_duplicates: How many times to duplicate the slab in the z direction
        make_symmetric: Whether to ensure the bottom row and top row are the same
    """

    # Make the 
    output = atoms.copy()
    output *= [1, 1, z_duplicates]

    # Append a vacuum layer in the Z direction
    original_z = output.cell[2, 2]
    output.center(vacuum=10, axis=2)

    # # Move the bottom row up
    if make_symmetric: 
        bottom_row = output[output.positions[:, 2] < output.positions[:, 2].min() + 0.1].copy()
        bottom_row.positions[:, 2] += original_z
        assert 'O' in bottom_row.symbols
        output.extend(bottom_row)

    # Recenter
    output.wrap()
    output.center(axis=2)
    
    return output

## Save each slab
Generate a slab for each of the relaxed structres in `cp2k-relaxed.db`

In [6]:
with connect('../0_relax-bulk/cp2k-relax.db') as db:
    for row in db.select(supercell=supercell):
        # Compile cell information
        atoms = row.toatoms(True)
        name = atoms.info['key_value_pairs']['name']

        # Check if already outputted
        output_name = f'{name}_{"".join(map(str, facet))}_{supercell}-cells_{layers}-layers'
        out_dir = Path('surfaces') / output_name
        out_dir.mkdir(parents=True, exist_ok=True)

        # Assemble the slabs
        term_a = slabify(atoms, repeats, True)
        
        term_b = atoms.copy()
        term_b.translate([0.0, 0.0, atoms.positions[1, 2] + 0.2])
        term_b.wrap()
        term_b = slabify(term_b, repeats, True)
        
        for term, slab in enumerate([term_a, term_b]):
            # Save the slab and site information
            slab_dir = out_dir / f'term={term}'
            if slab_dir.exists():
                continue
            slab_dir.mkdir()

            # WRite the unrelaxed coordiantes
            slab.write(slab_dir / 'unrelaxed.extxyz', columns=['symbols', 'positions'])